In [1]:
#
#  This notebook demonstrates the following:
#  * Use the Keras built-in Gemma instruction fine-tuned model ('gemma_instruct_2b_en')
#  * Experiments with ways to integrate conversation history
#

In [2]:
#  Python imports
#
#  Notes:
#  * Make sure you install the packages in requirements.txt
#  * Make sure you setup your KAGGLE secrets via env vars.
#
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
import kagglehub
from langchain.schema.runnable import Runnable
from typing import Any, Optional
import tensorflow as tf
from keras.config import disable_interactive_logging
import gc
from IPython.display import Markdown
import textwrap

In [16]:
# Useful functions
#
def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + text + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
def makebold(txt):
    return '\x1b[1;30m'+txt+'\x1b[0m'
def makeblue(txt):
    return '\x1b[1;34m'+txt+'\x1b[0m'
def makegreen(txt):
    return '\x1b[1;32m'+txt+'\x1b[0m'
def makered(txt):
    return '\x1b[1;33m'+txt+'\x1b[0m'

In [17]:
# Abstract interaction with the Gemma model
#
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    #print(makebold("Raw Prompt:\n") + makered("\"" + prompt + "\""))
    response = self.model.generate(prompt, max_length=1024)
    result = response.replace(prompt, "")  # Extract only the new response    
    #print(makebold("Raw Response:\n") + makeblue("\"" + response + "\""))
    self.add_to_history_as_model(result)
    return result

In [18]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_2b_en")
chat = ChatState(gemma_lm)

In [19]:
message = "My name is Daniel."
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>My name is Daniel.</blockquote></font><font size='+1' color='teal'>🤖<blockquote>> Hello Daniel! It's a pleasure to meet you. What would you like to talk about today?</blockquote></font>

In [7]:
message = "What is my name?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>What is my name?</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> I do not have access to personal information or user data, including names. Therefore, I am unable to provide information regarding your name.
</blockquote></font>

In [8]:
message = "Tell me a story."
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Tell me a story.</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> I would be happy to listen and tell a story if you wish. What kind of story would you like to hear?
</blockquote></font>

In [9]:
message = "Tell me a story about a dog."
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Tell me a story about a dog.</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> In a quaint village nestled amidst rolling hills, there lived an extraordinary dog named Sparky. With fur as black as night and eyes that sparkled like stars, he possessed a heart of gold and a playful spirit that brought joy to all who knew him.
> 
> One sunny afternoon, as Sparky bounded through the park, his tail wagging furiously, he stumbled upon a group of children playing. Their laughter and shouts caught his attention, and he couldn't resist joining in the fun. As he chased after a ball, he encountered a group of puppies who were too small to see. With gentle nudges and playful nips, he helped them up and encouraged them to play.
> 
> Through his unwavering loyalty and infectious enthusiasm, Sparky became the beloved pet of the village. Children and adults alike flocked to him for his companionship and wisdom. He taught them the importance of kindness, compassion, and the joy of giving.
> 
> And so, Sparky's legacy lived on, inspiring generations to come. His story spread far and wide, reminding everyone that even the smallest of creatures can make a big difference.
</blockquote></font>

In [10]:
message = "What is my name?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>What is my name?</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> I do not have access to personal information or user data, including names. Therefore, I am unable to provide information regarding your name.
</blockquote></font>

In [11]:
message = "Continue telling the story."
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Continue telling the story.</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> The sun began its golden descent, casting long shadows across the rolling hills. As the last rays of sunlight faded, a group of friends gathered around a crackling bonfire, their faces illuminated by the flickering flames.
> 
> "Let's tell a story," said Emily, her voice warm and inviting. "A story about a brave knight who saved the kingdom from a dark and evil sorcerer."
> 
> The friends listened intently, their eyes wide with anticipation. Emily began her tale, describing the knight's journey, his courage, and his unwavering determination. She spoke of his battles, his triumphs, and the sacrifices he made for the greater good.
> 
> As Emily's voice faded into the night, the friends were left with a sense of wonder and awe. They had never heard a story like this before, a story that combined adventure, magic, and the power of human spirit.
> 
> And so, the friends sat together, lost in the world of the story, their minds racing with possibilities and their hearts filled with inspiration.
</blockquote></font>

In [12]:
message = "What is a library?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>What is a library?</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> A library is a building or online resource center that contains a large collection of books, journals, and other printed materials, made available to the public for free or at a reduced cost. Libraries provide access to information, education, and entertainment to individuals from all walks of life.
> 
> Libraries offer a wide range of services, including:
> 
> - Lending books and other materials
> - Providing access to online resources
> - Hosting events and programs
> - Offering research assistance
> - Providing meeting and study spaces
> 
> Libraries play a vital role in promoting literacy, fostering a love of learning, and providing a space for community gathering.
</blockquote></font>

In [13]:
message = "Continue telling the story."
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Continue telling the story.</blockquote></font><font size='+1' color='teal'>🤖<blockquote>

> The library was a beacon of knowledge and inspiration, bathed in the warm glow of countless windows. Rows upon rows of towering bookshelves held stories of every kind, from ancient classics to modern bestsellers. The air was thick with the scent of old paper and the faintest hint of old wood.
> 
> As I wandered through the stacks, I was greeted by the warm smiles of librarians and the gentle murmur of conversations. I found a cozy corner to sit and lose myself in a book.
> 
> With each turn of the page, I was transported to different worlds and time periods. I discovered the joys of adventure in tales of knights and dragons, the wisdom of history in the chronicles of ancient civilizations, and the power of love in the stories of romance.
> 
> And as I sat there, lost in the pages, I felt a sense of peace and connection. The library was more than just a place to borrow books; it was a place where I could find solace, inspiration, and a community of readers.
</blockquote></font>

In [15]:
# Free up resources
chat = None
gemma_lm = None
gc.collect()

0